In [1]:
from datetime import datetime
from services.db_service import DbService

decks_collection = DbService().deck_collection()
tournaments_collection = DbService().events_collection()

In [2]:
def get_archetypes(limit=10, offset=0):
    date = datetime.strptime("2024-05-13", "%Y-%m-%d")
    archetypes = tournaments_collection.aggregate([{"$match": {"date": {"$gte": date}}},
                                             {"$project": {
                                                 "decklists": "$players.decklist"}},
                                             {"$unwind": "$decklists"},
                                             {"$group": {"_id": "$decklists"}},
                                             {"$lookup": {
                                                 "from": "decklists", "localField": "_id", "foreignField": "_id", "as": "decklists"}},
                                             {"$unwind": "$decklists"},
                                             {"$match": {
                                                 "decklists.banned": {"$ne": True}}},
                                             {"$group": {
                                                 "_id": "$decklists.name", "count": {"$sum": 1}}},
                                             {"$sort": {"count": -1}},
                                             {"$skip": offset}, {"$limit": limit},])

    return list(archetypes)

In [3]:
def get_minimum_decks(count):
    return round((count * 5)/100)


def get_archetype_pool(archetype, minimum_decks):
    date = datetime.strptime("2024-05-13", "%Y-%m-%d")
    pool = tournaments_collection.aggregate([{"$match": {"date": {"$gte": date}}},
                                             {"$project": {
                                                 "decklists": "$players.decklist"}},
                                             {"$unwind": "$decklists"},
                                             {"$group": {"_id": "$decklists"}},
                                             {"$lookup": {"from": "decklists", "localField": "_id",
                                                          "foreignField": "_id", "as": "decklists"}},
                                             {"$unwind": "$decklists"},
                                             {"$match": {"decklists.banned": {
                                                 "$ne": True}, "decklists.name": {"$eq": archetype}}},
                                             {"$project": {"_id": "$decklists._id", "cards": {
                                                 "$concatArrays": ["$decklists.maindeck", "$decklists.sideboard"]}}},
                                             {"$unwind": "$cards"},
                                             {"$group": {
                                                 "_id": {"_id": "$_id", "card": "$cards"}, "count": {"$sum": 1}}},
                                             {"$group": {"_id": "$_id.card", "count": {
                                                 "$max": "$count"}, "total_decks": {"$sum": 1}}},
                                             {"$lookup": {"from": "pool", "localField": "_id",
                                                          "foreignField": "name", "as": "pool"}},
                                             {"$unwind": "$pool"},
                                             {"$project": {"_id": 0, "card": "$pool.scryfall_name",
                                                           "count": 1, "total_decks": 1}},
                                             {"$lookup": {"from": "personal_pool", "localField": "card",
                                                          "foreignField": "scryfall_name", "as": "personal_pool"}},
                                             {"$unwind": {"path": "$personal_pool",
                                                          "preserveNullAndEmptyArrays": True}},
                                             {"$project": {"card": 1, "total_decks": 1, "count": {
                                                 "$subtract": ["$count", {"$ifNull": ["$personal_pool.count", 0]}]}}},
                                             {"$match": {"count": {"$gt": 0}, "total_decks": {
                                                 "$gte": minimum_decks}}},
                                             {"$sort": {"card": 1}}])

    return list(pool)

In [4]:

offset = 0
limit = 50
archetypes = get_archetypes(offset=offset, limit=limit)

for archetype in archetypes:
    file_name = f"{archetype["_id"]}.txt"

    pool = get_archetype_pool(
        archetype["_id"], get_minimum_decks(archetype["count"]))
    with open(f"../archetypes-wishlist/{file_name}", "w") as f:
        for card in pool:
            f.write(f"{card['count']} {card['card']}\n")